In [1]:
import re
import pickle
import nltk
from nltk.tokenize import  TweetTokenizer
from nltk.corpus import stopwords


In [8]:
from nltk.stem.snowball import SnowballStemmer

tokenizer =  TweetTokenizer()
stop = set(stopwords.words('english')+['<user>','<url>',',','-','.','[',']',':','&'])
stemmer = SnowballStemmer("english");
mojitable = pickle.load(open('emojtable.😃', 'rb'))
mojitable["< \ 3"] = "</3";
mojitable = {k: v for k, v in mojitable.items() if v != ""}
keys = [k for k, v in mojitable.items()]
prog = re.compile("(?:\s|^)((?:[^\s][^a-zA-Z1-9\n]+)(?:[^\s](?:[^a-zA-Z1-9\n]+|$))+)")
lineHead = re.compile("^(\d+,)");

def replaceMoji(s):
    def checkSubstring(emo):
        for k in keys:
            if k in emo :
                return mojitable[k]
        return "";
    splitted = prog.split(s)
    moj = ""
    nomoj = ""
    for i in range(0,int(len(splitted)/2)):
        nomoj += splitted[2*i]+" "
        emo = splitted[2*i+1].strip()
        parsedEmo = emo 
        if " " in parsedEmo:
            parsedEmo = mojitable.get(emo,checkSubstring(emo))
        moj += " "+parsedEmo+" "    
    return (moj,nomoj+splitted[-1]);
def prepareOneLine(line):
    try :
        tweet = line.decode('utf-8')
    except UnicodeDecodeError :
        tweet = str(line)[2:-5]
        print("cant parse "+tweet)
    head = lineHead.search(tweet)
    header = ""
    if head != None :
        header = head.group(1)
        tweet = tweet[len(header):]
    return (header+preparOneTweets(tweet)+"\n").encode('utf-8')
            
def preparOneTweets(tweet):
    moj,nomoj = replaceMoji(tweet)
    return  " ".join([stemmer.stem(i) for i in tokenizer.tokenize(nomoj) if i not in stop])+" "+moj
def preprocessFile(file):
    lines = []
    with open(file,'rb') as f :
        lines = map(prepareOneLine,f.readlines())
        f.close()
    with open("cleaned"+file,"wb") as f:
        for l in lines:
            f.write(l)
        f.close()
        print("done "+file)
("kerrri total forgot could use extra ticket gave away  ! ! !  ( ( ( ")
def checkSubstring(emo):
        for k in keys:
            if k in emo :
                return mojitable[k]
        return "";

In [9]:
files = ["train_neg.txt","train_neg_full.txt","train_pos.txt","train_pos_full.txt","test_data.txt"]
for f in files:
    preprocessFile(f)

cant parse rt allaah subhanau taa'la say ( interpret mean \x93 salaam ( peac ) upon ...
cant parse rt \x93 loool call #teamsomalia consid come ...
cant parse franco ( life \xd7 ) ( paperback francisco franco ( 1892-1975 ) grew quiet backwat el ferrol ...
cant parse franco ( life \xd7 ) ( paperback francisco franco ( 1892-1975 ) grew quiet backwat el ferrol ...
cant parse yes ! rt yah , pep \x93 gracia pep guardiola fabul 4 year 10 ...
done train_neg.txt
done train_neg_full.txt
done train_pos.txt
done train_pos_full.txt
done test_data.txt
